In [1]:
import sys
sys.path.append("/Users/olya/main/Centaur AI/NRN/torchlogic/nrn/")

In [2]:
from sklogic.classifiers import RNRNClassifier
# from nrn.models import BanditNRNClassifier

In [3]:
import warnings
warnings.filterwarnings("ignore", message="Choices for a categorical distribution should be a tuple")
warnings.filterwarnings("ignore", message="To copy construct from a tensor, it is recommended")
warnings.filterwarnings("ignore", message="IProgress not found")

import copy

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler

import optuna
import numpy as np
import pandas as pd
from scipy.special import softmax, expit

import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import calibration_curve, CalibrationDisplay

# from nrn.models import BanditNRNClassifier
# from nrn.utils.trainers import BanditNRNTrainer

# Load Data

In [4]:
data = load_breast_cancer()
data.target_names

array(['malignant', 'benign'], dtype='<U9')

In [5]:
data

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

# Preprocess Data

In [6]:
mms = MinMaxScaler()
X = mms.fit_transform(data.data)

In [7]:
y = data.target.reshape(-1, 1)

In [9]:
X_train_array, X_test_array, y_train_array, y_test_array = train_test_split(X, y, test_size=0.1, random_state=42)
X_train_array, X_val_array, y_train_array, y_val_array = train_test_split(X_train_array, y_train_array, test_size=0.22, random_state=42)

In [10]:
X_train = pd.DataFrame(X_train_array, columns=data.feature_names)
X_train.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0.264991,0.293879,0.249050,0.146554,0.282567,0.069873,0.004358,0.014533,0.321717,0.180918,...,0.198150,0.294776,0.175059,0.093123,0.215479,0.037789,0.004456,0.030144,0.185295,0.060803
1,0.287236,0.139669,0.268952,0.164199,0.278866,0.055119,0.010682,0.043882,0.198485,0.109941,...,0.241907,0.143390,0.216893,0.119740,0.200951,0.030251,0.014569,0.121375,0.106840,0.070576
2,0.221449,0.248901,0.206689,0.117709,0.207457,0.051899,0.019461,0.057753,0.308586,0.154802,...,0.190324,0.205757,0.165347,0.087815,0.187611,0.036354,0.024329,0.146460,0.229844,0.086646
3,0.197785,0.395671,0.187686,0.100445,0.443893,0.123919,0.020982,0.053479,0.280303,0.241786,...,0.157595,0.450426,0.143682,0.067047,0.376610,0.081022,0.028610,0.147973,0.278139,0.106192
4,0.279663,0.115320,0.263285,0.157964,0.152749,0.054935,0.052952,0.058201,0.139899,0.123210,...,0.202419,0.093817,0.184222,0.096294,0.165027,0.084806,0.094728,0.184399,0.146659,0.092549


In [11]:
y_train = pd.DataFrame(y_train_array)
y_train.head()

,0
0,1
1,1
2,1
3,1
4,1


In [12]:
X_val = pd.DataFrame(X_val_array, columns=data.feature_names)
X_test = pd.DataFrame(X_test_array, columns=data.feature_names)
X_val.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0.344503,0.336152,0.373437,0.206320,0.232373,0.625483,0.703608,0.387575,0.325253,0.584035,...,0.262184,0.312100,0.281837,0.128736,0.120980,0.380330,0.541773,0.517182,0.164203,0.348682
1,0.601022,0.404802,0.595052,0.445599,0.409317,0.351267,0.436504,0.548211,0.516162,0.151432,...,0.527215,0.603412,0.533841,0.332973,0.334346,0.309020,0.378115,0.774914,0.488863,0.158337
2,0.302854,0.710517,0.294036,0.175483,0.359484,0.168333,0.066050,0.162227,0.159091,0.214827,...,0.253291,0.788646,0.230490,0.127900,0.303969,0.111389,0.055935,0.266770,0.124384,0.142398
3,0.155190,0.232330,0.152443,0.075207,0.326262,0.187964,0.102109,0.121173,0.307576,0.361837,...,0.103166,0.267058,0.102943,0.042322,0.494816,0.191431,0.142412,0.286357,0.221959,0.260724
4,0.246533,0.121069,0.243867,0.132471,0.487226,0.232225,0.066893,0.143241,0.414141,0.310657,...,0.192458,0.154318,0.204044,0.088478,0.472363,0.224321,0.079193,0.281340,0.375320,0.245113


In [13]:
y_val = pd.DataFrame(y_val_array)
y_test = pd.DataFrame(y_test_array)
# y_val.head()

# Model training

In [14]:
clf = RNRNClassifier(binarization=False)
clf

In [15]:
clf.fit(X_train, y_train)

# Prediction

In [16]:
y_val_pred = clf.predict(X_val)

In [17]:
y_val_pred

,Class 0
0,0
1,0
2,1
3,1
4,1
...,...
108,0
109,0
110,0
111,1


In [18]:
y_val

,0
0,1
1,0
2,1
3,1
4,1
...,...
108,0
109,0
110,0
111,1


# Evaluation

In [19]:
roc_auc_score(y_val, y_val_pred)

0.8812292358803986

# Explanations

In [21]:
print(clf.explain_sample(X_val, sample_index=3, quantile=1, decision_boundary=0.5))

0: The prediction is in the Class 0 because: 


AND 
	The compactness error was less than 0.668 (66th percentile)
	The concavity error was less than 0.502 (50th percentile)
	The mean area was less than 0.266 (26th percentile)
	The mean perimeter was less than 0.394 (39th percentile)
	The mean radius was less than 0.413 (41th percentile)
	The mean symmetry was less than 0.655 (65th percentile)
	The worst area was less than 0.24 (24th percentile)
	The worst concave points was less than 0.487 (48th percentile)
	The worst perimeter was less than 0.24 (24th percentile)
	The worst radius was less than 0.351 (35th percentile)
	The worst symmetry was less than 0.556 (55th percentile)
